# Building interactive data visualization with Bokeh


* Data source: datacamp tidy data aquired from gapminder.
    

In [28]:
import pandas as pd

data = pd.read_csv('https://assets.datacamp.com/production/repositories/401/datasets/09378cc53faec573bcb802dce03b01318108a880/gapminder_tidy.csv', index_col='Year')
print(data.head())

          Country  fertility    life  population  child_mortality     gdp  \
Year                                                                        
1964  Afghanistan      7.671  33.639  10474903.0            339.7  1182.0   
1965  Afghanistan      7.671  34.152  10697983.0            334.1  1182.0   
1966  Afghanistan      7.671  34.662  10927724.0            328.7  1168.0   
1967  Afghanistan      7.671  35.170  11163656.0            323.3  1173.0   
1968  Afghanistan      7.671  35.674  11411022.0            318.1  1187.0   

          region  
Year              
1964  South Asia  
1965  South Asia  
1966  South Asia  
1967  South Asia  
1968  South Asia  


## EDA: 

In [29]:
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10111 entries, 1964 to 2006
Data columns (total 7 columns):
Country            10111 non-null object
fertility          10100 non-null float64
life               10111 non-null float64
population         10108 non-null float64
child_mortality    9210 non-null float64
gdp                9000 non-null float64
region             10111 non-null object
dtypes: float64(5), object(2)
memory usage: 631.9+ KB


(10111, 7)

## Bokeh Application

In [30]:
from bokeh.io import output_notebook, push_notebook, show
import numpy as np
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [31]:
source = ColumnDataSource(data={'x':data.loc[1970].fertility, 'y':data.loc[1970].life,'country':data.loc[1970].Country})

p = figure(title='1970', x_axis_label='Fertility (children per woman)', y_axis_label='Life Expectancy (years)',
           plot_height=300, plot_width=500,
           tools=[HoverTool(tooltips='@country')])

p.circle(x='x', y='y', source=source)

#show(p)

GlyphRenderer(id='2740', ...)

## jupyter interaction

In [39]:
from ipywidgets import interact
#import numpy as np

#from bokeh.io import push_notebook, show, output_notebook
#from bokeh.plotting import figure

## Starting the App

In [33]:
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, curdoc



source = ColumnDataSource(data={'x': data.loc[1970].fertility, 'y':data.loc[1970].life, 
                                 'country': data.loc[1970].Country, 'pop': (data.loc[1970].population/20000000)+2, 
                                 'region': data.loc[1970].region,})
 
                      
xmin,xmax=min(data.fertility),max(data.fertility)
ymin,ymax=min(data.life), max(data.life)

plot=figure(title='Gapminder Data for 1970', plot_height=500, plot_width=700, x_range=(xmin,xmax),y_range=(ymin,ymax))

plot.circle(x='x',y='y',fill_alpha = 0.8, source=source)

plot.xaxis.axis_label='Fertility (children per woman)'
plot.yaxis.axis_label='Life Expectancy(years)'

#curdoc().add_root(plot)
#curdoc().title='Gapminder'



## Enhancing the plot with some shading

In [34]:
# Make a list of unique values from the region column 

region_list=data.region.unique().tolist()

from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6

color_mapper = CategoricalColorMapper(factors=region_list, palette= Spectral6)
plot.circle(x='x',y='y', fill_alpha=0.8, source=source, color=dict(field='region',transform=color_mapper), legend='region')

plot.legend.location='top_right'

#Add plot to current document and add titlle

#curdoc().add_root(plot)
#curdoc().tittle='Gapminder'
#show(plot)

## Add Slider to vary the year

* Add the slider to the plot to change the year being plotted to create an updated plot() function 
and associate it with the slide to selct values from 1970 to 2010



## Customizing based on user input

In [35]:
# Import the necessary modules
from bokeh.layouts import widgetbox , row
from bokeh.models import Slider

# Define the callback function: update_plot
def update_plot(attr, old, new):
    # Assign the value of the slider: yr
    yr = slider.value
    # Set new_data
    new_data = {
        'x'       : data.loc[yr].fertility,
        'y'       : data.loc[yr].life,
        'country' : data.loc[yr].Country,
        'pop'     : (data.loc[yr].population / 20000000) + 2,
        'region'  : data.loc[yr].region,
    }
    # Assign new_data to: source.data
    source.data = new_data
    
    # Add title to figure: plot.title.text
    plot.title.text = 'Gapminder data for %d' % yr

# Make a slider object: slider
slider = Slider(title='Year', start=1970, end=2010, step=1, value=1970)

# Attach the callback to the 'value' property of slider
slider.on_change('value',update_plot)

# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = row(widgetbox(slider), plot)
#curdoc().add_root(layout)

#show(layout)
    

# Adding a Hover Tool


In [36]:
from bokeh.models import HoverTool

hover=HoverTool(tooltips=[('Country','@country')])
plot.add_tools(hover)
layout=row(widgetbox(slider),plot)
#curdoc().add_root(layout)

# Adding drop down to the app

In [41]:
from bokeh.models import Select


def update_plot(attr, old, new):
    # Assign the value of the slider: yr
    # Read current value of slider and 2 dropdowns    
    yr = slider.value
    x=x_select.value
    y=y_select.value
    #label axis as plot
    plot.xaxis.axis_label=x
    plot.yaxis.axis_label=y
    
    # Set new_data
    new_data = {
        'x'       : data.loc[yr].fertility,
        'y'       : data.loc[yr].life,
        'country' : data.loc[yr].Country,
        'pop'     : (data.loc[yr].population / 20000000) + 2,
        'region'  : data.loc[yr].region,
    }
    # Assign new_data to: source.data
    source.data = new_data
    #set the range of all axes
    plot.x_range.start=min(data[x])
    plot.x_range.end=max(data[x])
    plot.y_range.start=min(data[y])
    plot.x_range.end=max(data[y])
    
    # Add title to figure: plot.title.text
    plot.title.text = 'Gapminder data for %d' % yr
    
    push_notebook()

# Make a slider object: slider
slider = Slider(title='Year', start=1970, end=2010, step=1, value=1970)

# Attach the callback to the 'value' property of slider
slider.on_change('value',update_plot)

# create a dropdown
x_select=Select(options=['fertility', 'life', 'child_mortality', 'gdp'], value='fertility', title='x_axisdata')
x_select.on_change('value',update_plot)
y_select=Select(options=['fertility','life', 'child_mortality','gdp'], value='fertility',title='y_axisdata')
y_select.on_change('value',update_plot)

# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = row(widgetbox(slider, x_select, y_select), plot)
curdoc().add_root(layout)

show(layout, notebook_handle=True)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html

